In [ ]:
import os
from google.colab import userdata
from IPython.display import Image, display
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq

In [ ]:
# Install required libraries
!pip install langgraph langsmith
!pip install langchain langchain_groq langchain_community

In [ ]:
# Retrieve API keys
groq_api_key = userdata.get('groq_api_key')
langsmith_api_key = userdata.get('LANGSMITH_API_KEY')

In [ ]:
# Print API keys (for verification, can be removed later)
print(langsmith_api_key)
print(groq_api_key)

In [ ]:
# Set environment variables
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "CourseLanggraph"

In [ ]:
# Initialize the language model
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

In [ ]:
# Define State class
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Create StateGraph
graph_builder = StateGraph(State)
def chatbot(state: State):
    return {"messages": llm.invoke(state['messages'])}

In [ ]:
# Add nodes and edges to graph
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Compile graph
graph = graph_builder.compile()

In [ ]:
# Display graph
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:

# Chatbot loop
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("Good Bye")
        break
    
    for event in graph.stream({'messages': ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"].content)
